In [1]:
import numpy as np
import pandas as pd
import pgeocode
import plotly.graph_objects as go

In [2]:
data21 = pd.read_csv("../Data/2021/R13351471_SL860.csv",header=[0])

In [23]:
df21 = data21.copy()
df21.head()

,Geo_QName,SE_A00001_001,SE_A00002_001,SE_A00002_002,SE_A00002_003,SE_A00003_001,SE_A00003_002,SE_A00003_003,SE_A02001_001,SE_A02001_002,...,SE_A10065_001,SE_A10065_002,SE_A10066_001,SE_A10066_002,SE_A10066_003,SE_A10066_004,SE_A10066_005,SE_A10066_006,SE_A10066_007,SE_A10066_008
0,ZCTA5 01001,16088,16088,1424.6630,11.292497,12.114230,11.292500,0.821730,16088,7996,...,2526,783,6791,2195,2496,947,750,314,62,27
1,ZCTA5 01002,27323,27323,513.0521,53.255802,54.908740,53.255800,1.652938,27323,13344,...,3349,888,9985,3164,2675,2074,1502,436,97,37
2,ZCTA5 01003,13711,13711,17050.8700,0.804123,0.809529,0.804123,0.005406,13711,6666,...,0,0,15,0,15,0,0,0,0,0
3,ZCTA5 01005,4947,4947,114.5428,43.189077,43.446380,43.189080,0.257308,4947,2249,...,750,227,1761,250,785,368,233,109,0,16
4,ZCTA5 01007,15304,15304,278.9168,54.869406,57.547950,54.869410,2.678543,15304,7668,...,2960,414,5917,1202,2312,818,959,502,124,0


## Preprocess Data

In [24]:
df21.insert(0, 'zip_code', df21[('Geo_QName')].astype(str).str[5:])

In [25]:
df21

,zip_code,Geo_QName,SE_A00001_001,SE_A00002_001,SE_A00002_002,SE_A00002_003,SE_A00003_001,SE_A00003_002,SE_A00003_003,SE_A02001_001,...,SE_A10065_001,SE_A10065_002,SE_A10066_001,SE_A10066_002,SE_A10066_003,SE_A10066_004,SE_A10066_005,SE_A10066_006,SE_A10066_007,SE_A10066_008
0,01001,ZCTA5 01001,16088,16088,1424.663000,11.292497,12.114230,11.292500,0.821730,16088,...,2526,783,6791,2195,2496,947,750,314,62,27
1,01002,ZCTA5 01002,27323,27323,513.052100,53.255802,54.908740,53.255800,1.652938,27323,...,3349,888,9985,3164,2675,2074,1502,436,97,37
2,01003,ZCTA5 01003,13711,13711,17050.870000,0.804123,0.809529,0.804123,0.005406,13711,...,0,0,15,0,15,0,0,0,0,0
3,01005,ZCTA5 01005,4947,4947,114.542800,43.189077,43.446380,43.189080,0.257308,4947,...,750,227,1761,250,785,368,233,109,0,16
4,01007,ZCTA5 01007,15304,15304,278.916800,54.869406,57.547950,54.869410,2.678543,15304,...,2960,414,5917,1202,2312,818,959,502,124,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33637,99923,ZCTA5 99923,13,13,0.749552,17.343694,17.344510,17.343690,0.000817,13,...,0,0,13,13,0,0,0,0,0,0
33638,99925,ZCTA5 99925,917,917,17.099770,53.626460,62.449470,53.626460,8.823010,917,...,161,66,345,95,104,37,33,60,7,9
33639,99926,ZCTA5 99926,1445,1445,10.881210,132.797749,261.667600,132.797700,128.869800,1445,...,263,113,457,80,145,77,69,35,26,25
33640,99927,ZCTA5 99927,11,11,2.350756,4.679345,7.293139,4.679345,2.613793,11,...,0,0,11,11,0,0,0,0,0,0


In [26]:
df21 = df21.loc[pd.to_numeric(df21.zip_code).between(10000,20000)] # Filtering rows to limit the data we work on. Selecting group of states starting with 1

In [27]:
df21.set_index('zip_code', inplace=True)

In [28]:
df21.shape

(3726, 2187)

## Extracting required columns for analysis

In [29]:
table_names = ['SE_B13004', 'SE_B18002', 'SE_A03001', 'SE_A13007', 'SE_A14028', 'SE_A14024_', 'SE_B12001', 'SE_A12003', 'SE_A12004', 'SE_A17005', 'SE_A17009', 'SE_A20001', 'SE_A00002']
matching_columns = [col for col in df21.columns.get_level_values(0) if col.startswith(tuple(table_names))]
df21 = df21.loc[:, df21.columns.get_level_values(0).isin(matching_columns)]

In [30]:
df21 = df21.dropna()

## Income

### Below Poverty Line

In [31]:
df21.loc[:, df21.columns.get_level_values(0).str.startswith('SE_A13007')]

,SE_A13007_001,SE_A13007_002,SE_A13007_003,SE_A13007_004,SE_A13007_005,SE_A13007_006,SE_A13007_007,SE_A13007_008,SE_A13007_009,SE_A13007_010,SE_A13007_011
zip_code,,,,,,,,,,,
10001,23125,2841,786,624,162,2055,20284,16773,16266,507,3511
10002,66733,16542,3249,2465,784,13293,50191,33060,31112,1948,17131
10003,41773,4755,1086,654,432,3669,37018,30132,29397,735,6886
10004,3810,113,19,18,1,94,3697,3245,3193,52,452
10005,7668,303,267,143,124,36,7365,6970,6862,108,395
...,...,...,...,...,...,...,...,...,...,...,...
19973,20442,3034,1511,1092,419,1523,17408,11754,11215,539,5654
19975,8676,590,260,189,71,330,8086,4121,3896,225,3965
19977,17668,1700,781,673,108,919,15968,10835,10378,457,5133


In [32]:
df21['incomeBelowPovertyLevelProportion'] = ((df21[( 'SE_A13007_002')].values) /(df21[('SE_A13007_001')]))
df21.incomeBelowPovertyLevelProportion.describe()
df21 = df21.loc[: ,~df21.columns.get_level_values(0).str.startswith('SE_A13007')] # Drop the columns from the table after processing


### Per Capita

In [33]:
df21.loc[: ,df21.columns.get_level_values(0).str.startswith('SE_A14024_001')]

,SE_A14024_001
zip_code,
10001,94177.0
10002,37557.0
10003,106585.0
10004,147878.0
10005,128970.0
...,...
19973,30349.0
19975,46887.0
19977,31613.0


In [34]:
df21.loc[:, 'incomePerCapita'] = df21.loc[: ,df21.columns.get_level_values(0).str.startswith('SE_A14024_001')].values
df21.incomePerCapita.describe()
df21 = df21.loc[: ,~df21.columns.get_level_values(0).str.startswith('SE_A14024')] # Drop the columns from the table after processing

### Gini Index

In [35]:
df21.loc[:, 'incomeGiniIndex'] = df21.loc[: ,df21.columns.get_level_values(0).str.startswith('SE_A14028_001')] # Drop the columns from the table after processing
df21 = df21.loc[: ,~df21.columns.get_level_values(0).str.startswith('SE_A14028')] # Drop the columns from the table after processing
df21.incomeGiniIndex.describe()

count    3572.000000
mean        0.404293
std         0.084709
min         0.000400
25%         0.368600
50%         0.408700
75%         0.450525
max         0.807100
Name: incomeGiniIndex, dtype: float64

In [36]:
df21.shape

(3572, 68)

# Processing Y Variable

Same as in 2018 preprocessing of Y variable for 2021 also we consider : 

The number of families for each zip code are available for poverty ratios less than 1, between 1 and 2 and more than 2.
All the families with ratio under 2 are considered to be poor or struggling and over 2 are considered to be ok.  
  
Based on this information, we convert this information into one column which has values 0 or 1  
0 - if more than 50% of families in the zip code have a ratio over 2  
1 - if 50% or more of families in the zip code are having ratio under 2  

In [37]:
df21.loc[:, df21.columns.get_level_values(0).str.startswith('SE_B13004')]

,SE_B13004_001,SE_B13004_002,SE_B13004_003,SE_B13004_004,SE_B13004_005
zip_code,,,,,
10001,26023,3557,1993,5550,20473
10002,76047,20554,16375,36929,39118
10003,44965,4872,2478,7350,37615
10004,4794,141,79,220,4574
10005,8637,303,95,398,8239
...,...,...,...,...,...
19973,25302,4173,4551,8724,16578
19975,9609,667,1229,1896,7713
19977,22937,2402,2145,4547,18390


In [38]:
df21['underPoverty'] = np.where(df21[( 'SE_B13004_005')].div(df21[('SE_B13004_001')]) < 0.5,1,0) # F13004 table contains the number of families within the different ratio ranges
df21 = df21.loc[: ,~df21.columns.get_level_values(0).str.startswith('SE_F13004')] # Drop the columns from the table after processing

In [39]:
len(df21.loc[df21['underPoverty'] == 1]) # There are 277 zip codes in region 1 which have more than 50% of families that are poor or struggling

255

In [40]:
fig = go.Figure(data=go.Scattergeo(
        lon = pgeocode.Nominatim('us').query_postal_code(df21.index.tolist()).longitude,
        lat = pgeocode.Nominatim('us').query_postal_code(df21.index.tolist()).latitude,
        marker_color = df21['underPoverty'],
        ))
fig.update_layout(
        title = 'Poverty levels of zip codes',
        geo_scope='usa',
    )
fig.show()

## Race and Population

In [41]:
df21.head(2)

,SE_A00002_001,SE_A00002_002,SE_A00002_003,SE_A03001_001,SE_A03001_002,SE_A03001_003,SE_A03001_004,SE_A03001_005,SE_A03001_006,SE_A03001_007,...,SE_B13004_005,SE_A20001_001,SE_A20001_002,SE_A20001_003,SE_A20001_004,SE_A20001_005,incomeBelowPovertyLevelProportion,incomePerCapita,incomeGiniIndex,underPoverty
zip_code,,,,,,,,,,,,,,,,,,,,,
10001,26966,43227.06,0.623822,26966,16963,2117,40,4527,50,1091,...,20473,26966,800,26166,7241,21172,0.122854,94177.0,0.5524,0
10002,76807,93406.05,0.822291,76807,24687,6453,624,30288,19,8856,...,39118,76361,4255,72106,41115,35755,0.247883,37557.0,0.5922,0


In [42]:
df21.loc[:, 'popDensity'] = df21.loc[:,('SE_A00002_002')].values 
# df = df.loc[: ,~df.columns.get_level_values(1).str.startswith('SE_A00001', 'SE_A00002', 'SE_A00003')] # Drop the columns from the table after processing
df21 = df21.loc[: ,~df21.columns.get_level_values(0).str.startswith('SE_A00001')] # Drop the columns from the table after processing

df21.popDensity.describe()

count      3572.000000
mean       3528.333910
std       12728.910457
min           0.130871
25%          61.214317
50%         229.790400
75%        1372.559250
max      177218.800000
Name: popDensity, dtype: float64

In [43]:
df21.drop(['SE_A00002_001','SE_A00002_003', ],axis= 1 , inplace= True)

### Finding white majority:

We check if white-only population is the majority in a zip code  
Primarily non-white areas might be experiencing higher levels of poverty.  
0 - if less than 50% of population in the zip code are white  
1 - if more than 50% of families in the zip code are non-white

In [44]:
df21.drop(df21.loc[df21.loc[:,df21.columns.get_level_values(0).str.startswith('SE_A03001_001')].isin([0]).any(axis = 1)].index, inplace = True)
df21.loc[:, 'whiteMajority'] = np.where((df21.loc[:,'SE_A03001_002'].values) / (df21.loc[:,'SE_A03001_001'].values) > 0.5,1,0) # If the proportion of white alone population is more than 50% we set this binary variable to 1
df21 = df21.loc[: ,~df21.columns.get_level_values(0).str.startswith('SE_A03001')] # Drop the columns from the table after processing
df21['whiteMajority'].describe()

count    3572.000000
mean        0.935050
std         0.246471
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: whiteMajority, dtype: float64

## Education

### Highest degree attainment

In [45]:
df21.loc[: ,df21.columns.get_level_values(0).str.startswith('SE_B12001_')]

,SE_B12001_001,SE_B12001_002,SE_B12001_003,SE_B12001_004
zip_code,,,,
10001,21296,1538,4182,15576
10002,60747,19576,19073,22098
10003,37589,1360,5267,30962
10004,3624,13,112,3499
10005,6829,6,494,6329
...,...,...,...,...
19973,17963,2481,12269,3213
19975,8079,719,4325,3035
19977,18937,1670,13003,4264


In [46]:
df21 = df21.loc[df21['SE_B12001_001'] != 0] # Some of the zip codes have 0. To avoid RuntimeWarning we drop these rows

In [47]:
df21.loc[:, 'eduHsOrLessProportion'] = ((df21.loc[:,('SE_B12001_002')].values) / (df21.loc[:,('SE_B12001_001')].values)) # Proportion of 25yo and over with less than high school education
df21.loc[:, 'eduHsProportion'] = ((df21.loc[:,('SE_B12001_003')].values) / (df21.loc[:,('SE_B12001_001')].values)) # Proportion of 25yo and over with less than high school education
df21.loc[:, 'eduBachOrBetterProportion'] = ((df21.loc[:,('SE_B12001_004')].values) / (df21.loc[:,('SE_B12001_001')].values)) # Proportion of 25yo and over with less than high school education
df21 = df21.loc[: ,~df21.columns.get_level_values(0).str.startswith('SE_B12001_')] # Drop the columns from the table after processing
print(df21.eduHsOrLessProportion.describe())
print(df21.eduHsProportion.describe())
print(df21.eduBachOrBetterProportion.describe())

count    3572.000000
mean        0.094539
std         0.076540
min         0.000000
25%         0.046494
50%         0.080146
75%         0.121506
max         1.000000
Name: eduHsOrLessProportion, dtype: float64
count    3572.000000
mean        0.619442
std         0.166229
min         0.000000
25%         0.535327
50%         0.651717
75%         0.729847
max         1.000000
Name: eduHsProportion, dtype: float64
count    3572.000000
mean        0.286019
std         0.182457
min         0.000000
25%         0.158206
50%         0.239594
75%         0.375125
max         1.000000
Name: eduBachOrBetterProportion, dtype: float64


## School Enrollment

In [48]:
df21.loc[: ,df21.columns.get_level_values(0).str.startswith('SE_A12004')] # Drop the columns from the table after processing

,SE_A12004_001,SE_A12004_002,SE_A12004_003
zip_code,,,
10001,26310,4375,21935
10002,75343,13024,62319
10003,53328,15093,38235
10004,4326,571,3755
10005,8292,839,7453
...,...,...,...
19973,24657,5322,19335
19975,9506,1223,8283
19977,25049,5658,19391


In [49]:
df21.loc[:, 'eduEnrollmentMajority'] = np.where((df21.loc[:,('SE_A12004_002')].values) / (df21.loc[:,('SE_A12004_001')].values) > 0.5,1,0) # If the proportion of 3yo and over enrolled in school is more than 50% we set this binary variable to 1
df21 = df21.loc[: ,~df21.columns.get_level_values(0).str.startswith('SE_A12004')] # Drop the columns from the table after processing
df21['eduEnrollmentMajority'].value_counts()
df21

,SE_A00002_002,SE_A12003_001,SE_A12003_002,SE_A12003_003,SE_A12003A_001,SE_A12003A_002,SE_A12003A_003,SE_A12003B_001,SE_A12003B_002,SE_A12003B_003,...,incomeBelowPovertyLevelProportion,incomePerCapita,incomeGiniIndex,underPoverty,popDensity,whiteMajority,eduHsOrLessProportion,eduHsProportion,eduBachOrBetterProportion,eduEnrollmentMajority
zip_code,,,,,,,,,,,,,,,,,,,,,
10001,43227.0600,826,0,826,265,0,265,561,0,561,...,0.122854,94177.0,0.5524,0,43227.0600,1,0.072220,0.196375,0.731405,0
10002,93406.0500,2076,33,2043,1163,27,1136,913,6,907,...,0.247883,37557.0,0.5922,0,93406.0500,0,0.322255,0.313974,0.363771,0
10003,95234.9600,7257,17,7240,3064,6,3058,4193,11,4182,...,0.113830,106585.0,0.5532,0,95234.9600,1,0.036181,0.140121,0.823698,0
10004,10525.1200,72,0,72,12,0,12,60,0,60,...,0.029659,147878.0,0.4566,0,10525.1200,1,0.003587,0.030905,0.965508,0
10005,118529.5000,47,0,47,0,0,0,47,0,47,...,0.039515,128970.0,0.3579,0,118529.5000,1,0.000879,0.072339,0.926783,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19973,305.0022,1113,73,1040,556,73,483,557,0,557,...,0.148420,30349.0,0.4386,0,305.0022,1,0.138117,0.683015,0.178868,0
19975,376.3289,220,0,220,140,0,140,80,0,80,...,0.068004,46887.0,0.4447,0,376.3289,1,0.088996,0.535339,0.375665,0
19977,311.7339,838,89,749,548,59,489,290,30,260,...,0.096219,31613.0,0.4006,0,311.7339,1,0.088187,0.686645,0.225168,0


### School Dropout Rate

In [50]:
df21 = df21.loc[df21['SE_A12003_001'] != 0] # Some of the zip codes have 0 civilian population. To avoid RuntimeWarning we drop these rows

In [51]:
df21.loc[:, 'eduDropoutMajority'] = np.where((df21.loc[:,('SE_A12003_002')].values) / (df21.loc[:,('SE_A12003_001')].values) > 0.5,1,0) # If the proportion of late teens dropped out from school is more than 50% we set this binary variable to 1
df21 = df21.loc[: ,~df21.columns.get_level_values(0).str.startswith('SE_A12003')] # Drop the columns from the table after processing
df21.eduDropoutMajority.value_counts()

C:\Users\jayav\AppData\Local\Temp\ipykernel_14812\2230501799.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



eduDropoutMajority
0    3162
1      51
Name: count, dtype: int64

## Employment

In [52]:
#Proportion of labor force employment

df21.loc[: ,df21.columns.get_level_values(0).str.startswith('SE_A17005')]

,SE_A17005_001,SE_A17005_002,SE_A17005_003,SE_A17005A_001,SE_A17005A_002,SE_A17005A_003,SE_A17005B_001,SE_A17005B_002,SE_A17005B_003
zip_code,,,,,,,,,
10001,17828,17098,730,9085,8672,413,8743,8426,317
10002,36431,33689,2742,18809,17437,1372,17622,16252,1370
10003,34455,32693,1762,17586,16925,661,16869,15768,1101
10004,3264,3211,53,1913,1912,1,1351,1299,52
10005,7237,7005,232,3586,3459,127,3651,3546,105
...,...,...,...,...,...,...,...,...,...
19973,13265,12307,958,6751,6130,621,6514,6177,337
19975,4381,4085,296,2263,2096,167,2118,1989,129
19977,11616,11051,565,5213,4876,337,6403,6175,228


In [53]:
df21.loc[:, 'employedProportion'] = ((df21.loc[:,('SE_A17005_002')].values) / (df21.loc[:,('SE_A17005_001')].values))
df21 = df21.loc[: ,~df21.columns.get_level_values(0).str.startswith('SE_A17005')] # Drop the columns from the table after processing
df21.employedProportion.describe()

count    3213.000000
mean        0.945058
std         0.047810
min         0.200000
25%         0.931515
50%         0.952280
75%         0.970732
max         1.000000
Name: employedProportion, dtype: float64

### Employment by sector IN PROGRESS

In [54]:
df21.loc[: ,df21.columns.get_level_values(0).str.startswith('SE_A17009')]

,SE_A17009_001,SE_A17009_002,SE_A17009_003,SE_A17009_004,SE_A17009_005,SE_A17009_006
zip_code,,,,,,
10001,17098,12421,1028,2287,1362,0
10002,33689,23553,3459,3752,2776,149
10003,32693,22385,1790,4263,4160,95
10004,3211,2115,255,632,173,36
10005,7005,5713,482,408,402,0
...,...,...,...,...,...,...
19973,12307,8098,1598,1510,1046,55
19975,4085,2645,692,430,310,8
19977,11051,6900,2663,741,747,0


In [55]:
df21.loc[:, 'employedPvtSecProportion'] = ((df21.loc[:,('SE_A17009_002')].values) / (df21.loc[:,('SE_A17009_001')].values))
df21.loc[:, 'employedPubSecProportion'] = ((df21.loc[:,('SE_A17009_003')].values) / (df21.loc[:,('SE_A17009_001')].values))
df21.loc[:, 'employedSelfEmpProportion'] = ((df21.loc[:,('SE_A17009_004')].values) / (df21.loc[:,('SE_A17009_001')].values))
df21.loc[:, 'employedPvtNonProProportion'] = ((df21.loc[:,('SE_A17009_005')].values) / (df21.loc[:,('SE_A17009_001')].values))
df21.loc[:, 'employedUnpaidFamProportion'] = ((df21.loc[:,('SE_A17009_006')].values) / (df21.loc[:,('SE_A17009_001')].values))
df21 = df21.loc[: ,~df21.columns.get_level_values(0).str.startswith('SE_A17009')] # Drop the columns from the table after processing


## Health

In [56]:
df21.loc[: ,df21.columns.get_level_values(0).str.startswith('SE_A20001')]

,SE_A20001_001,SE_A20001_002,SE_A20001_003,SE_A20001_004,SE_A20001_005
zip_code,,,,,
10001,26966,800,26166,7241,21172
10002,76361,4255,72106,41115,35755
10003,54273,1636,52637,9993,46799
10004,4795,51,4744,243,4607
10005,8637,83,8554,469,8153
...,...,...,...,...,...
19973,25368,1844,23524,11688,16004
19975,9583,719,8864,4775,6751
19977,22637,1927,20710,8743,15621


In [57]:
df21 = df21.loc[df21.loc[:,('SE_A20001_001')] != 0] # Some of the zip codes have 0. To avoid RuntimeWarning we drop these rows
df21.loc[:, 'insuredProportion'] = ((df21.loc[:,('SE_A20001_003')].values) / (df21.loc[:,('SE_A20001_001')].values))
df21 = df21.loc[: ,~df21.columns.get_level_values(0).str.startswith('SE_A20001')] # Drop the columns from the table after processing
df21.insuredProportion.describe()

count    3213.000000
mean        0.944553
std         0.057151
min         0.393746
25%         0.933110
50%         0.958146
75%         0.975775
max         1.000000
Name: insuredProportion, dtype: float64

In [58]:
df21.shape

(3213, 27)

## Housing

### Proportion of renters spending more than 30% of income on rent

In [59]:
df21.loc[:, df21.columns.get_level_values(0).str.startswith('SE_B18002')]

,SE_B18002_001,SE_B18002_002,SE_B18002_003
zip_code,,,
10001,10493,2366,1708
10002,28887,7294,7312
10003,15936,3197,3370
10004,1249,215,188
10005,3987,877,512
...,...,...,...
19973,2365,549,636
19975,632,248,73
19977,1677,409,409


In [61]:
df21 = df21.loc[df21.loc[:, ( 'SE_B18002_001')] != 0] # Remove observations with 0 to avoid error while dividing
df21.loc[:, 'housingRentMoreThan30pcProportion'] = ((df21.loc[:,('SE_B18002_002')].values + df21.loc[:,('SE_B18002_003')].values ) / (df21.loc[:,('SE_B18002_001')].values))
df21 = df21.loc[: ,~df21.columns.get_level_values(0).str.startswith('SE_B18002')] # Drop the columns from the table after processing


In [62]:
df21.shape

(3140, 25)

IndexError: Too many levels: Index has only 1 level, not 2